In [23]:
import cv2
import numpy as np
import os

In [24]:
video_folder = "/home/persie/Videos/mokap/"
session_id = os.listdir(video_folder)
video_file = []
for folder in session_id:
    files = os.listdir(os.path.join(video_folder, folder))
    for file in files:
        video_file.append(os.path.join(video_folder, folder, file))


In [25]:
cap = cv2.VideoCapture(video_file[33])
cap.set(cv2.CAP_PROP_POS_FRAMES, 134)
ret, frame = cap.read()
clear_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

In [56]:
def draw_circle(event,x,y,flags,param):
    #Because of the use of global variables, they must be defined before using them, outside of any functions
    global radius, text_image, mask  
    if flags == cv2.EVENT_FLAG_SHIFTKEY:
            cv2.circle(mask,(x,y),radius,(0,0,0),-1)    
    if flags == cv2.EVENT_FLAG_LBUTTON:
        cv2.circle(mask,(x,y),radius,(255,255,255),-1)      
        
    text_image = cv2.rectangle(text_image, (0, 70), (35, 40), (255, 255, 255), -1)    
    text_image = cv2.putText(text_image, str(radius), (0, 65), cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.8, color=(0, 0, 255), thickness=2)         
        

In [47]:

np.where(mask == 1)


(array([ 525,  646,  704,  707,  707,  715,  725,  727,  729,  742,  745,
         764,  793,  793,  799,  803,  813,  814,  815,  819,  828,  836,
         838,  840,  846,  847,  851,  863,  868,  870,  877,  881,  886,
         887,  888,  892,  897,  903,  911,  914,  920,  924,  940,  952,
         956,  966,  969,  976,  986, 1008, 1026, 1047, 1056]),
 array([430, 461, 474, 408, 467, 471, 437, 437, 430, 416, 413, 366, 521,
        522, 514, 531, 452, 535, 529, 484, 530, 395, 539, 505, 544, 545,
        548, 556, 554, 551, 521, 524, 551, 529, 515, 547, 517, 520, 534,
        551, 551, 461, 531, 551, 548, 509, 546, 362, 541, 539, 536, 533,
        531]))

In [ ]:
import cv2
import numpy as np
import argparse
 
 #Select the image pixels and the mask
img_object = cv.imread(cv.samples.findFile(args.input1), cv.IMREAD_GRAYSCALE)
img_scene = cv.imread(cv.samples.findFile(args.input2), cv.IMREAD_GRAYSCALE)
if img_object is None or img_scene is None:
    print('Could not open or find the images!')
    exit(0)
 
#-- Step 1: Detect the keypoints using SURF Detector, compute the descriptors
minHessian = 400
detector = cv.xfeatures2d_SURF.create(hessianThreshold=minHessian)
keypoints_obj, descriptors_obj = detector.detectAndCompute(img_object, None)
keypoints_scene, descriptors_scene = detector.detectAndCompute(img_scene, None)
 
#-- Step 2: Matching descriptor vectors with a FLANN based matcher
# Since SURF is a floating-point descriptor NORM_L2 is used
matcher = cv.DescriptorMatcher_create(cv.DescriptorMatcher_FLANNBASED)
knn_matches = matcher.knnMatch(descriptors_obj, descriptors_scene, 2)
 
#-- Filter matches using the Lowe's ratio test
ratio_thresh = 0.75
good_matches = []
for m,n in knn_matches:
    if m.distance < ratio_thresh * n.distance:
        good_matches.append(m)
 
#-- Draw matches
img_matches = np.empty((max(img_object.shape[0], img_scene.shape[0]), img_object.shape[1]+img_scene.shape[1], 3), dtype=np.uint8)
cv.drawMatches(img_object, keypoints_obj, img_scene, keypoints_scene, good_matches, img_matches, flags=cv.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
 
#-- Localize the object
obj = np.empty((len(good_matches),2), dtype=np.float32)
scene = np.empty((len(good_matches),2), dtype=np.float32)
for i in range(len(good_matches)):
    #-- Get the keypoints from the good matches
    obj[i,0] = keypoints_obj[good_matches[i].queryIdx].pt[0]
    obj[i,1] = keypoints_obj[good_matches[i].queryIdx].pt[1]
    scene[i,0] = keypoints_scene[good_matches[i].trainIdx].pt[0]
    scene[i,1] = keypoints_scene[good_matches[i].trainIdx].pt[1]
 
H, _ =  cv.findHomography(obj, scene, cv.RANSAC)
 
#-- Get the corners from the image_1 ( the object to be "detected" )
obj_corners = np.empty((4,1,2), dtype=np.float32)
obj_corners[0,0,0] = 0
obj_corners[0,0,1] = 0
obj_corners[1,0,0] = img_object.shape[1]
obj_corners[1,0,1] = 0
obj_corners[2,0,0] = img_object.shape[1]
obj_corners[2,0,1] = img_object.shape[0]
obj_corners[3,0,0] = 0
obj_corners[3,0,1] = img_object.shape[0]
 
scene_corners = cv.perspectiveTransform(obj_corners, H)
 
#-- Draw lines between the corners (the mapped object in the scene - image_2 )
cv.line(img_matches, (int(scene_corners[0,0,0] + img_object.shape[1]), int(scene_corners[0,0,1])),\
    (int(scene_corners[1,0,0] + img_object.shape[1]), int(scene_corners[1,0,1])), (0,255,0), 4)
cv.line(img_matches, (int(scene_corners[1,0,0] + img_object.shape[1]), int(scene_corners[1,0,1])),\
    (int(scene_corners[2,0,0] + img_object.shape[1]), int(scene_corners[2,0,1])), (0,255,0), 4)
cv.line(img_matches, (int(scene_corners[2,0,0] + img_object.shape[1]), int(scene_corners[2,0,1])),\
    (int(scene_corners[3,0,0] + img_object.shape[1]), int(scene_corners[3,0,1])), (0,255,0), 4)
cv.line(img_matches, (int(scene_corners[3,0,0] + img_object.shape[1]), int(scene_corners[3,0,1])),\
    (int(scene_corners[0,0,0] + img_object.shape[1]), int(scene_corners[0,0,1])), (0,255,0), 4)
 
#-- Show detected matches
cv.imshow('Good Matches & Object detection', img_matches)
 
cv.waitKey()

In [63]:
#Select frame with object in
cap = cv2.VideoCapture(video_file[33])
cap.set(cv2.CAP_PROP_POS_FRAMES, 1507)
ret, frame = cap.read()
object_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

In [115]:
# Run the mask creation code
text_image = object_frame
mask = np.zeros_like(object_frame, np.uint8)
# Set the initial circle radius
radius = 50
line_1 = "Click with the left mouse button to add, double click to remove"
line_2 = "Up = radius + 5,"
line_3 = "Down = radius - 5"
text_image = cv2.putText(text_image, line_1, (10, 10), cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.5, color=(0, 255, 0), thickness=2)  
text_image = cv2.putText(text_image, line_2, (10, 22), cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.5, color=(0, 255, 0), thickness=2)  
text_image = cv2.putText(text_image, line_3, (10, 34), cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.5, color=(0, 255, 0), thickness=2)  
combined_image = cv2.bitwise_or(text_image, mask)
cv2.namedWindow('Mask On Frame')
cv2.imshow('Mask On Frame', combined_image)
cv2.setMouseCallback('Mask On Frame',draw_circle)


while(1):
    combined_image = cv2.bitwise_or(text_image, mask)
    cv2.imshow('Mask On Frame',combined_image)
    cv2.imshow('Mask',mask)
    k = cv2.waitKey(20) & 0xFF
    if k == ord("q"):
        break
    if k == 82:
        #upkey
        radius = radius + 5
    if k == 84:
        #downkey
        radius = radius - 5

cap.release()
cv2.destroyAllWindows()

In [116]:
#save the mask separately
object_mask = mask

In [128]:
# Get another test frame
cap = cv2.VideoCapture(video_file[33])
cap.set(cv2.CAP_PROP_POS_FRAMES, 2934)
ret, frame = cap.read()
img_scene = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

In [129]:
#Select the image pixels and the mask and just check that it is alright
img_object = cv2.bitwise_and(object_frame, object_mask)

if img_object is None or img_scene is None:
    print('Could not open or find the images!')
    exit(0)

cv2.imshow("Masked Image", img_object)
cv2.imshow("Comparison Scene", img_scene)
while 1:
    if cv2.waitKey(20) & 0xFF == ord("q"):
        break
cap.release()
cv2.destroyAllWindows()



In [133]:
#-- Step 1: Detect the keypoints using SIFT Detector, compute the descriptors
detector = cv2.SIFT_create()
keypoints_obj, descriptors_obj = detector.detectAndCompute(img_object, None)
keypoints_scene, descriptors_scene = detector.detectAndCompute(img_scene, None)
 

In [134]:
#-- Step 2: Matching descriptor vectors with a FLANN based matcher
# Since SURF is a floating-point descriptor NORM_L2 is used
matcher = cv2.DescriptorMatcher_create(cv2.DescriptorMatcher_FLANNBASED)
knn_matches = matcher.knnMatch(descriptors_obj, descriptors_scene, 2)


#-- Filter matches using the Lowe's ratio test
ratio_thresh = 0.75
good_matches = []
for m,n in knn_matches:
    if m.distance < ratio_thresh * n.distance:
        good_matches.append(m)
 
#-- Draw matches
img_matches = np.empty((max(img_object.shape[0], img_scene.shape[0]), img_object.shape[1]+img_scene.shape[1], 3), dtype=np.uint8)
cv2.drawMatches(img_object, keypoints_obj, img_scene, keypoints_scene, good_matches, img_matches, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)


error: OpenCV(4.10.0) /home/conda/feedstock_root/build_artifacts/libopencv_1723431247556/work/modules/flann/src/miniflann.cpp:336: error: (-210:Unsupported format or combination of formats) in function 'buildIndex_'
> type=0
> 

In [132]:

#-- Localize the object
obj = np.empty((len(good_matches),2), dtype=np.float32)
scene = np.empty((len(good_matches),2), dtype=np.float32)
for i in range(len(good_matches)):
    #-- Get the keypoints from the good matches
    obj[i,0] = keypoints_obj[good_matches[i].queryIdx].pt[0]
    obj[i,1] = keypoints_obj[good_matches[i].queryIdx].pt[1]
    scene[i,0] = keypoints_scene[good_matches[i].trainIdx].pt[0]
    scene[i,1] = keypoints_scene[good_matches[i].trainIdx].pt[1]
 
H, _ =  cv2.findHomography(obj, scene, cv2.RANSAC)
 
#-- Get the corners from the image_1 ( the object to be "detected" )
obj_corners = np.empty((4,1,2), dtype=np.float32)
obj_corners[0,0,0] = 0
obj_corners[0,0,1] = 0
obj_corners[1,0,0] = img_object.shape[1]
obj_corners[1,0,1] = 0
obj_corners[2,0,0] = img_object.shape[1]
obj_corners[2,0,1] = img_object.shape[0]
obj_corners[3,0,0] = 0
obj_corners[3,0,1] = img_object.shape[0]
 
scene_corners = cv2.perspectiveTransform(obj_corners, H)
 
#-- Draw lines between the corners (the mapped object in the scene - image_2 )
cv2.line(img_matches, (int(scene_corners[0,0,0] + img_object.shape[1]), int(scene_corners[0,0,1])),\
    (int(scene_corners[1,0,0] + img_object.shape[1]), int(scene_corners[1,0,1])), (0,255,0), 4)
cv2.line(img_matches, (int(scene_corners[1,0,0] + img_object.shape[1]), int(scene_corners[1,0,1])),\
    (int(scene_corners[2,0,0] + img_object.shape[1]), int(scene_corners[2,0,1])), (0,255,0), 4)
cv2.line(img_matches, (int(scene_corners[2,0,0] + img_object.shape[1]), int(scene_corners[2,0,1])),\
    (int(scene_corners[3,0,0] + img_object.shape[1]), int(scene_corners[3,0,1])), (0,255,0), 4)
cv2.line(img_matches, (int(scene_corners[3,0,0] + img_object.shape[1]), int(scene_corners[3,0,1])),\
    (int(scene_corners[0,0,0] + img_object.shape[1]), int(scene_corners[0,0,1])), (0,255,0), 4)
 
#-- Show detected matches
cv2.imshow('Good Matches & Object detection', img_matches)
cv2.waitKey()
cap.release()
cv2.destroyAllWindows()

In [114]:
cv2.imshow('Good Matches & Object detection', img_matches)
cv2.waitKey()
cap.release()
cv2.destroyAllWindows()
